In [5]:
import pandas as pd

In [2]:
cols = ['Material', 'Movement type', 'Qty in unit of entry', 'Unit of Entry', 'Order']

In [3]:
df = pd.concat(pd.read_excel(
    '../data/TONG.XLSX', sheet_name=None, usecols=cols ,converters={
            'Material': str,
            'Order': str
        }
    ), ignore_index=True
)


In [7]:
df = df.loc[
    df['Movement type'].isin([101, 102, 261, 262]) &
    ~pd.isna(df['Order'])
]

In [8]:
sx = df.loc[df['Movement type'].isin([101, 102])]
nvl = df.loc[df['Movement type'].isin([261, 262])]

In [9]:
sx = sx.groupby(
    ['Material', 'Unit of Entry', 'Order']).agg(
        {'Qty in unit of entry': 'sum'}
)
sx.reset_index(inplace=True)
sx.rename(columns={
    'Material': 'Product',
    'Unit of Entry': 'Product Unit',
    'Qty in unit of entry': 'Product Quantity'
}, inplace=True)
sx = sx.loc[sx['Product Quantity'] != 0]

nvl = nvl.groupby(
    ['Material', 'Unit of Entry', 'Order']).agg(
        {'Qty in unit of entry': 'sum'}
)
nvl.reset_index(inplace=True)
nvl.rename(columns={
    'Unit of Entry': 'Material Unit',
    'Qty in unit of entry': 'Material Quantity'
}, inplace=True)


In [10]:
nvl['Material Quantity'] = -nvl['Material Quantity']

In [11]:
sx.drop_duplicates(inplace=True)
result = pd.merge(sx, nvl, how="left", on='Order', sort=False,
                          indicator=False, validate=None)

In [12]:
result = result.groupby(['Product', 'Product Unit', 'Material', 'Material Unit']).agg(
            {'Product Quantity': 'sum', 'Material Quantity': 'sum'})

In [13]:
result.reset_index(inplace=True)
result['BOM'] = result['Material Quantity'] / result['Product Quantity']

In [15]:
import pandas as pd
df = pd.concat(pd.read_excel(
    '../data/2022-03-15_BOM_KG3.xlsx', sheet_name=None,
    ), ignore_index=True
)

In [16]:
# null = df[df['Usage'].notna()]
df['Usage'] = df['Usage'].fillna(0)
df.to_excel('../output/KG3.xlsx')

In [2]:
df = pd.read_excel('../data/BOM winnercom.xlsx', header=6, usecols=['Mã mẹ', 'Mã con', 'Actual BOM'], converters={'Actual BOM': float})

In [28]:
df

,Mã mẹ,Mã con,Actual BOM
0,96271L8000,NaN,0.0
1,96271L8000,W11,1.005746
2,96271L8000,W14,1.132497
3,96271L8000,W01,1.007474
4,96271L8000,W05,1.005183
...,...,...,...
3342,717-368483-00006,W04,1.0
3343,717-368483-00006,W20,1.0
3344,717-368483-00006,W14,6.0
3345,717-368483-00006,W26,0.0


In [3]:
result = df.groupby(['Mã mẹ', 'Mã con'], sort=False).agg(
            {'Actual BOM': 'sum'})

In [5]:
result.to_excel('../output/winnercom.xlsx')

In [1]:
import pandas as pd

df = pd.read_excel('../data/BOM 2021.xlsx', header=11)
cols = df.columns[18:]
df = df[cols]
df.set_index('Material Name', inplace=True)
trans = df.transpose()
cols = trans.columns
trans.reset_index(inplace=True)
trans = trans.iloc[2:]
cols = [x  for x in trans.columns if not pd.isna(x)]
trans = trans[cols]

trans.melt(id_vars=["index"], ignore_index = False).to_excel('../output/hope.xlsx')
result= trans.set_index(["index"]).stack()
result = result.reset_index()
df.reset_index(inplace=True)
df = df[['Material Name', 'Unit.1']]
df
end = pd.merge(
    result,
    df,
    how="left",
    on='Material Name',
    sort=False,
    indicator=False,
    validate=None,
)
end
end = end.drop(end[end[0] == 0].index)
end.to_excel('../output/hope.xlsx')
end.to_excel('../output/hope.xlsx')
